In [2]:
#default_exp haystack_search

In [3]:
#export
import pprint
import numpy as np
import pandas as pd
import requests
import torch
from sklearn import metrics
from nltk import tokenize
from operator import itemgetter

import haystack
from haystack import database
import haystack.database.memory

from haystack.retriever.dense import EmbeddingRetriever
from pytorch_hackathon import rss_feeds

import seaborn as sns

In [4]:
import logging
logging.disable(logging.CRITICAL)

In [5]:
pd.set_option('max_colwidth', 100)

In [6]:
cm = sns.light_palette("green", as_cmap=True)

In [6]:
%cd ..

/home/piotr/Documents/pytorch_hackathon


In [7]:
!ls data

feeds.txt  topics.txt


In [8]:
rss_feed_urls = list(pd.read_table('data/feeds.txt', header=None).iloc[:,0].values)

In [9]:
feed_df = rss_feeds.get_feed_df(rss_feed_urls)

100%|██████████| 16/16 [00:08<00:00,  2.00it/s]
/home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py:63: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 63 of the file /home/kuba/Projects/pytorch_hackathon/pytorch_hackathon/rss_feeds.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  feed_df['text'] = feed_df['summary'].apply(lambda s: bs4.BeautifulSoup(s).text)


In [10]:
use_gpu = torch.cuda.is_available()

In [13]:
feed_df.head()

title  \
0  Adversarial Attack and Defense Strategies for Deep Speaker Recognition Systems   
1                    Knowledge Transfer via Dense Cross-Layer Mutual-Distillation   
2                     Neutral Face Game Character Auto-Creation via PokerFace-GAN   
3                                 AP-Loss for Accurate One-Stage Object Detection   
4        DeepGIN: Deep Generative Inpainting Network for Extreme Image Inpainting   

                                                                                          title_detail  \
0  {'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...   
1  {'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...   
2  {'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...   
3  {'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...   
4  {'type': 'text/plain', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/p...   

                                                                                                 links  \
0  [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/adversarial...   
1  [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/knowledge-t...   
2  [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/neutral-fac...   
3  [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/ap-loss-for...   
4  [{'rel': 'alternate', 'type': 'text/html', 'href': 'https://paperswithcode.com/paper/deepgin-dee...   

                                                                             link  \
0  https://paperswithcode.com/paper/adversarial-attack-and-defense-strategies-for   
1       https://paperswithcode.com/paper/knowledge-transfer-via-dense-cross-layer   
2  https://paperswithcode.com/paper/neutral-face-game-character-auto-creation-via   
3          https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object   
4     https://paperswithcode.com/paper/deepgin-deep-generative-inpainting-network   

                                                                                               summary  \
0  Robust speaker recognition, including in the presence of malicious attacks, is becoming increasi...   
1  Knowledge Distillation (KD) based methods adopt the one-way Knowledge Transfer (KT) scheme in wh...   
2  Besides, the neural network based renderer used in previous methods is also difficult to be exte...   
3  This paper alleviates this issue by proposing a novel framework to replace the classification ta...   
4  Existing image inpainting approaches usually encounter difficulties in completing the missing pa...   

                                                                                        summary_detail  \
0  {'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...   
1  {'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...   
2  {'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...   
3  {'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...   
4  {'type': 'text/html', 'language': None, 'base': 'https://us-east1-ml-feeds.cloudfunctions.net/pw...   

                                                                               id  \
0  https://paperswithcode.com/paper/adversarial-attack-and-defense-strategies-for   
1       https://paperswithcode.com/paper/knowledge-transfer-via-dense-cross-layer   
2  https://paperswithcode.com/paper/neutral-face-game-character-auto-creation-via   
3          https://paperswithcode.com/paper/ap-loss-for-accurate-one-stage-object   
4     https://paperswithcode.com/paper/deepgin-deep-generative-inpainting-network

In [14]:
#export


class Searcher:
    
    def __init__(
        self,
        model_name,
        text_col,
        use_gpu,
        max_document_length=256,
        quantize_model=True,
        document_store_cls=database.memory.InMemoryDocumentStore
    ):
        self.text_col = text_col
        self.embedding_col = text_col + '_emb'
        self.max_document_length = max_document_length
        self.model_name = model_name
        self.document_store = document_store_cls(
            embedding_field=self.embedding_col,
        )
        self.retriever = self._setup_retriever(use_gpu, quantize_model)

    def _setup_retriever(self, use_gpu, quantize_model):
        retriever = EmbeddingRetriever(
            document_store=self.document_store,
            embedding_model=self.model_name,
            use_gpu=use_gpu)
        if not use_gpu and quantize_model:
            self.set_quantized_model(retriever)
            
        return retriever

    def add_texts(
        self,
        df
    ):
        truncated_texts = [
            ' '.join(tokenize.wordpunct_tokenize(text)[:self.max_document_length])
            for text in df[self.text_col] 
        ]
        article_embeddings = self.retriever.embed_queries(
            texts=truncated_texts
        )

        df[self.embedding_col] = article_embeddings
        self.document_store.write_documents(df.to_dict(orient='records'))
    
    def search(self, query, top_k=10, **kwargs):
        if type(self.document_store) is database.elasticsearch.ElasticsearchDocumentStore:
            querying_fn = self.document_store.query 
        elif type(self.document_store) is database.memory.InMemoryDocumentStore:
            querying_fn = self.retriever.retrieve
        return querying_fn(
            query,
            top_k=top_k,
            **kwargs
        )
    
    @classmethod
    def set_quantized_model(cls, retriever):
        quantized_model = torch.quantization.quantize_dynamic(
            retriever.embedding_model.model,
            {torch.nn.Linear}, dtype=torch.qint8
        )
        retriever.embedding_model.model = quantized_model
        
    @classmethod 
    def sigmoid(cls, x):
        return 1 / (1 + np.exp(-x))
    
    @classmethod
    def doc_to_dict(cls, doc):
        d = {}
        d['text'] = doc.text
        d['title'] = doc.meta['title']
        d['score'] = doc.query_score
        d['link'] = doc.meta['link']
        d['date'] = doc.meta['date']
        return d

    def get_topic_score_df(self, raw_results, topic_strings):
        topic_query_strings = [
            'text is about {}'.format(topic)
            for topic in topic_strings
        ]

        results = [
            self.doc_to_dict(doc)
            for doc in raw_results 
        ]
        result_embeddings = np.array([
            doc.meta['text_emb']
            for doc in raw_results
        ]).astype('float32')
        topic_query_embeddings = np.array(self.retriever.embed_passages(
            list(topic_strings)
        )).astype('float32')

        scores_df = pd.DataFrame({})
        scores_df['title'] = list(map(itemgetter('title'), results))
        scores_df['text'] = list(map(itemgetter('text'), results))
        scores_df['link'] = list(map(itemgetter('link'), results))
        scores_df['date'] = list(map(itemgetter('date'), results))

        scores = pd.DataFrame(metrics.pairwise.cosine_similarity(
            result_embeddings,
            topic_query_embeddings
        ))
        scores.columns = topic_strings

        scores_df = pd.concat(
            [scores_df, self.sigmoid(scores)],
            axis=1
        )
        return scores_df

In [ ]:
[
    "deepset/sentence_bert",
    "sentence-transformers/bert-base-nli-mean-tokens",
    "sentence-transformers/bert-large-nli-mean-tokens",
    "sentence-transformers/bert-base-nli-max-tokens",
    "sentence-transformers/bert-base-nli-stsb-mean-tokens",
    "sentence-transformers/distilbert-base-nli-stsb-mean-tokens",
    "sentence-transformers/roberta-base-nli-stsb-mean-tokens",
    "sentence-transformers/roberta-large-nli-stsb-mean-tokens",
    "sentence-transformers/bert-base-nli-cls-token",
    "sentence-transformers/distilbert-base-nli-stsb-quora-ranking",
    "sentence-transformers/bert-large-nli-cls-token",
    "sentence-transformers/bert-large-nli-max-tokens"
]

In [13]:
model_name = "deepset/sentence_bert"

In [14]:
searcher = Searcher(
    model_name,
    'text',
    use_gpu=use_gpu
)

In [15]:
searcher.add_texts(feed_df[['text', 'title', 'link']])

Inferencing Samples: 100%|██████████| 74/74 [00:10<00:00,  7.19 Batches/s]
/etc/conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [16]:
article_texts = feed_df['text']

In [17]:
topic_strings = pd.read_table('data/topics.txt', header=None).iloc[:,0].values

In [18]:
print('\n'.join(topic_strings))

deep learning
natural language processing
computer vision
statistics
implementation
visualization
industry
software engineering
reddit question
arxiv
cloud computing
deployment
competitions
business
business intelligence


In [19]:
topic_query_strings = [
    'text is about {}'.format(topic)
    for topic in topic_strings
]

In [20]:
raw_results = searcher.search(
    topic_query_strings[1]
)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 31.56 Batches/s]


In [21]:
scores_df = searcher.get_topic_score_df(raw_results, topic_strings)

Inferencing Samples: 100%|██████████| 4/4 [00:00<00:00,  7.63 Batches/s]


In [22]:
scores_df.head().style.background_gradient(cmap=cm)